In [11]:
test_semantic_model_url = 'https://raw.githubusercontent.com/djouallah/Fabric_Notebooks_Demo/refs/heads/main/SemanticModel/sm/semantic_model.txt' 
SF = 0.1
output_dir="/tmp/llm"

In [12]:
import requests
import json
import duckdb
import pathlib
import time
import pandas as pd
import re
from   openai import AzureOpenAI, OpenAIError
import datetime

In [13]:
if SF <1 :
 schema = f"{str(SF).replace('.', '_')}"
else :
 schema = f'DS{SF:02d}'
import os
folder_path = '/some_location'
os.makedirs(folder_path, exist_ok=True)
db_path = folder_path +"/"+ schema +".duckdb"
if not pathlib.Path(db_path).exists():
    con = duckdb.connect(db_path)
    con.sql("SET memory_limit = '14GB' ")
    con.sql(f"CALL dsdgen(sf={SF})")
    con.close()
con = duckdb.connect()
con.sql(f""" attach or replace '{db_path}' as ds(read_only) ; use ds """)

In [14]:
def get_ai_response(user_message, LLM, provider, api_key=None, azure_endpoint=None, azure_api_version=None):
    try:
        github_response = requests.get(test_semantic_model_url)
        github_response.raise_for_status()
        system_prompt = github_response.text.strip()
    except requests.RequestException as e:
        return f"Error fetching system prompt from GitHub: {e}"

    generated_text = None

    if provider == "ollama":
        url = 'http://127.0.0.1:11434/api/chat'
        headers = {'Content-Type': 'application/json'}
        payload = {
            'model': LLM,
            'messages': [
                {'role': 'system', 'content': system_prompt},
                {'role': 'user', 'content': user_message}
            ],
            'stream': False
        }
        try:
            response = requests.post(url, headers=headers, json=payload)
            response.raise_for_status()
            data = response.json()
            generated_text = data.get('message', {}).get('content', '')
            if generated_text:
                generated_text = generated_text.replace("<think>", "").replace("</think>", "")
        except requests.RequestException as e:
            return f"Error with Ollama API request: {e}"
        except (KeyError, IndexError, AttributeError) as e:
            return f"Unexpected response format from Ollama: {e}\nFull response data: {data}"

    elif provider == "azure_foundry":
        if not azure_endpoint or not azure_api_version:
            return "Azure provider requires 'azure_endpoint' and 'azure_api_version' parameters."

        try:
            client = AzureOpenAI(
                api_version=azure_api_version,
                azure_endpoint=azure_endpoint,
                api_key=api_key,
            )

            response = client.chat.completions.create(
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_message}
                ],
                model=LLM,
            )

            generated_text = response.choices[0].message.content

        except OpenAIError as e:
            return f"Azure OpenAI API Error: {e}"
        except Exception as e:
            return f"An unexpected error occurred with Azure OpenAI: {e}"

    else:
        return f"Unsupported provider for this test setup: {provider}. Supported providers are 'ollama' and 'azure_foundry'."

    if generated_text is None:
        return f"Could not retrieve generated text from {provider} response."

    # --- Begin SQL/Markdown Cleaning ---
    cleaned_text = re.sub(
        r'```(sql|duckdb)?\s*([\s\S]*?)\s*```',
        r'\2',
        generated_text,
        flags=re.IGNORECASE
    ).strip()

    if "SELECT " in cleaned_text.upper() or "WITH " in cleaned_text.upper() or "```" in cleaned_text:
        cleaned_text = cleaned_text.replace("```sql", "").replace("```duckdb", "").replace("```", "").strip()

    return cleaned_text


In [15]:
def execute_sql_with_retry(query, test_model,provider, api_key, azure_endpoint, azure_api_version,max_attempts):
    attempt = 1
    current_query = query.strip()

    while attempt <= max_attempts:
        try:
            # Execute the SQL query
            result = con.sql(current_query).df()
            # Return the DataFrame result and the number of attempts
            return result, attempt
        except Exception as e:
            # Catch any exception during execution
            error_message = str(e).lower()

            # Check if the error is likely a syntax, parser, or binder error
            if "syntax" not in error_message and "parser" not in error_message and "binder" not in error_message:
                # If it's a non-syntax error, return the error message and attempts
                return f"Non-syntax error: {e}", attempt

            # If it's a syntax-like error, print attempt info
            print(current_query)
            print(f"Attempt {attempt}/{max_attempts} failed with syntax error: {e}")

            # If max attempts reached, return the final error message and attempts
            if attempt == max_attempts:
                return f"Max attempts reached. Last error: {e}", attempt

            # Prepare message for Gemini to fix the query
            message = (
                f"The following SQL query has a syntax error: '{current_query}'.\n"
                f"Error message: {e}\n"
                f"Please provide the corrected SQL query. Return only the corrected query without explanation."
            )

            # Get corrected query from Gemini
            corrected_query = get_ai_response(message, test_model,provider, api_key, azure_endpoint, azure_api_version)

            # If Gemini returns an error, return that error message and attempts
            if corrected_query.startswith("Error"):
                return f"Failed to get corrected query : {corrected_query}", attempt

            #print(f"Received corrected query")
            # Update current_query with the corrected one and increment attempt
            current_query = corrected_query.strip()
            attempt += 1
    return "Unexpected error or loop termination", attempt



In [16]:
def ask_question(questions, test_model,provider, azure_endpoint=None, azure_api_version=None, api_key=None):
    results_data = [] 
    for i, x in enumerate(questions):
        print(f"Question {i+1}: {x}") # Keep or remove print as needed
        start_time = time.time()
        sql_query_or_error = get_ai_response(x, test_model, provider, api_key, azure_endpoint, azure_api_version)
        print(sql_query_or_error) # Keep or remove print as needed
        query_result_data_json = [] # Initialize as an empty list for JSON result
        attempts_count = None
        error_details = None # Initialize error_details
        if sql_query_or_error is None or sql_query_or_error.startswith("Error"):
            # If get_ai_response returned an error or None, store the error string
            error_message = sql_query_or_error if sql_query_or_error is not None else "AI response was None"
            # print(f"Failed to get query from AI: {error_message}") # Keep or remove print
            # print("Execution: SKIPPED (AI error)") # Keep or remove print
            error_details = f"AI Error: {error_message}" # Store error details
            # query_result_data_json remains empty []
            result_row_count = 0 # Result count is 0 on error
        else:
            # Attempt to execute the SQL query
            # Ensure execute_sql_with_retry returns a DataFrame on success or a string error
            # Pass the test_model to execute_sql_with_retry as it might need it
            result_from_execution, attempts_count = execute_sql_with_retry(sql_query_or_error, test_model,provider,api_key, azure_endpoint, azure_api_version, max_attempts=3)

            display(result_from_execution)
            is_successful = isinstance(result_from_execution, pd.DataFrame)

            if is_successful:
                print("Execution: SUCCESS") # Keep or remove print
                query_result_data_json = result_from_execution.to_dict('records')
                error_details = None # No error details on success
                result_row_count = len(result_from_execution) # Calculate row count
            else:
                print("Execution: FAILED") # Keep or remove print
                #query_result_data_json remains empty []
                error_details = f"Execution Error: {result_from_execution}" # Store error details
                result_row_count = 0 # Result count is 0 on failure


        end_time = time.time()
        duration = round(end_time - start_time, 2)
        print(f"\nExecution Time: {duration:.2f} seconds") # Keep or remove print
        print(f" ############################### ") # Keep or remove print
        results_data.append({
            "model" : test_model,
            "nbr": i + 1,
            "question": x,
            "duration_s": duration, # Changed key name slightly for JSON convention
            "sql_query": sql_query_or_error, # Store the generated query string (could be AI error)
            "attempts": attempts_count,
            "result": query_result_data_json, # This is now a list of dictionaries
            "result_count": result_row_count, # Use the calculated row count
            "error_details": error_details # Store error details
        })

    os.makedirs(output_dir, exist_ok=True)
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    output_filename = f"results_{timestamp}.json"
    output_path = os.path.join(output_dir, output_filename)
    try:
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(results_data, f, indent=4)
        return f"Successfully processed {len(questions)} questions. Results saved to {output_path}"
    except IOError as e:
        return f"Error saving results to {output_path}: {e}"
    except Exception as e:
        return f"An unexpected error occurred during file saving: {e}"

In [17]:
questions = [
    # Easy Questions:
    "What is the overall total sales revenue?",
    "What is the total number of items sold across all transactions?",
    "What is the total monetary value of all returned items?",
    "List the names of all stores, ordered alphabetically by store name.",
    "What are the different item categories available? Order alphabetically by category name.",

    # Medium Questions:
    "Show total sales revenue for each year, ordered chronologically by year.",
    "Which store generated the most total sales revenue? (To ensure a deterministic result if there's a tie, you might order by revenue descending and then store name alphabetically).",
    "What is the total quantity of items sold, broken down by item brand? Order by quantity sold descending, and then by brand name alphabetically for ties.",
    "Compare total sales from preferred customers versus non-preferred customers, ordered by customer type.",
    "What is the total return amount for each city where stores are located, ordered alphabetically by city name.",

    # Hard Questions:
    "What is the net sales for each store name, ordered alphabetically by store name.",
    "Calculate the return rate for each item category, ordered alphabetically by item category name.",
    "What is the monthly trend of net sales during the year 2001, ordered chronologically by month.",
    "Which customer birth country exhibits the highest average return rate? (To ensure a deterministic result in case of ties, order by average return rate descending and then by country name alphabetically).",
    "List all item product names that have a return rate greater than 5% and their total sales, ordered by item product name alphabetically.",

    # Very Hard Questions:
    "For each store, what was the percentage change in net sales between two consecutive recent years (e.g., 2001 and 2002), ordered alphabetically by store name.",
    "What is the return rate for items sold on weekends versus weekdays, broken down by customer age groups (e.g., under 30, 30-45, over 45), ordered by age group and then by weekday/weekend indicator.",
    "Which item brand has shown the largest decrease in its return rate when comparing the average rate of 2001 to 2002, specifically for stores located in the 'TN' state? (Order by the decrease in return rate descending, and then by brand name alphabetically for ties).",
    "For each item class, compare the average net sales value per sales transaction between preferred and non-preferred customers. Order by average net sales descending, and then by item class name alphabetically for ties.",
    "Identify stores whose net sales are below 99% of the average net sales for stores in their respective countries, and list their manager, city, and current overall return rate, ordered alphabetically by country, then city, then store name."
]

In [18]:
%%time
ask_question(questions,"gpt-4o","azure_foundry", os.getenv("llm_endpoint"),"2025-01-01-preview" ,os.getenv("LLM_API_KEY"))

Question 1: What is the overall total sales revenue?
-- overall total sales revenue
SELECT 
    SUM(ss.ss_sales_price * ss.ss_quantity) AS total_sales_revenue
FROM store_sales AS ss;


,total_sales_revenue
0,5.143009e+08


Execution: SUCCESS

Execution Time: 2.11 seconds
 ############################### 
Question 2: What is the total number of items sold across all transactions?
-- What is the total number of items sold across all transactions?
SELECT
    SUM(ss.ss_quantity) AS total_quantity_sold
FROM store_sales AS ss;


,total_quantity_sold
0,13939520.0


Execution: SUCCESS

Execution Time: 1.98 seconds
 ############################### 
Question 3: What is the total monetary value of all returned items?
-- Total value of all returned items
SELECT 
    SUM(sr.sr_return_amt) AS total_returns
FROM store_returns AS sr;


...

,item_class,preferred_customer_flag,avg_net_sales_per_transaction
0,earings,None,2532.885873
1,custom,None,2363.180714
2,monitors,None,2272.177463
3,self-help,None,2269.849091
4,entertainments,None,2174.550328
...,...,...,...
396,womens,None,0.000000
397,womens watch,None,0.000000
398,None,Y,0.000000
399,None,N,0.000000


Execution: SUCCESS

Execution Time: 5.83 seconds
 ############################### 
Question 20: Identify stores whose net sales are below 99% of the average net sales for stores in their respective countries, and list their manager, city, and current overall return rate, ordered alphabetically by country, then city, then store name.
-- Identify stores whose net sales are below 99% of the average net sales for stores in their respective countries,
-- and list their manager, city, and current overall return rate, ordered alphabetically by country, city, then store name.

WITH StoreNetSales AS (
    -- Calculate net sales for each store
    WITH store_sales_agg AS (
        SELECT
            st.s_store_sk,
            st.s_country,
            st.s_store_name,
            SUM(ss.ss_sales_price * ss.ss_quantity) AS total_sales
        FROM store_sales AS ss
        INNER JOIN store AS st ON ss.ss_store_sk = st.s_store_sk
        GROUP BY st.s_store_sk, st.s_country, st.s_store_name
    ),

'Max attempts reached. Last error: Binder Error: Referenced table "sr" not found!\nCandidate tables: "ss"'

Execution: FAILED

Execution Time: 31.62 seconds
 ############################### 
CPU times: total: 15.5 s
Wall time: 2min 30s


'Successfully processed 20 questions. Results saved to /tmp/llm\\results_20250508_230344.json'

In [19]:
%%time
ask_question(questions,'qwen3:4b',"ollama")

Question 1: What is the overall total sales revenue?
SELECT
  total_sales
FROM (
  SELECT
    SUM(ss.ss_sales_price * ss.ss_quantity) AS total_sales
  FROM store_sales AS ss
) AS sales; -- overall total sales revenue


,total_sales
0,5.143009e+08


Execution: SUCCESS

Execution Time: 12.98 seconds
 ############################### 
Question 2: What is the total number of items sold across all transactions?
SELECT
  SUM(ss.ss_quantity) AS total_quantity
FROM
  store_sales ss; -- Question: What is the total number of items sold across all transactions?


,total_quantity
0,13939520.0


Execution: SUCCESS

Execution Time: 2.37 seconds
 ############################### 
Question 3: What is the total monetary value of all returned items?
SELECT
  SUM(sr.sr_return_amt) AS total_return_value
FROM store_returns sr; -- Total monetary value of all returned items


...

,i_class,preferred_avg_net_sales,non_preferred_avg_net_sales
0,accent,0.0,0.0
1,accent,0.0,0.0
2,accent,0.0,0.0
3,accent,0.0,0.0
4,accent,0.0,0.0
...,...,...,...
1795,None,0.0,0.0
1796,None,0.0,0.0
1797,None,0.0,0.0
1798,None,0.0,0.0


Execution: SUCCESS

Execution Time: 36.53 seconds
 ############################### 
Question 20: Identify stores whose net sales are below 99% of the average net sales for stores in their respective countries, and list their manager, city, and current overall return rate, ordered alphabetically by country, then city, then store name.
SELECT
  s.s_store_name,
  s.s_manager,
  s.s_city,
  (COALESCE(r.total_returns, 0) / NULLIF(COALESCE(s.total_sales, 0), 0)) * 100 AS return_rate
FROM (
  SELECT
    st.s_store_name,
    st.s_store_sk,
    st.s_store_name,
    st.s_city,
    st.s_country,
    SUM(ss.ss_sales_price * ss.ss_quantity) AS total_sales
  FROM store_sales AS ss
  INNER JOIN store AS st ON ss.ss_store_sk = st.s_store_sk
  GROUP BY st.s_store_sk
) AS s
FULL OUTER JOIN (
  SELECT
    st.s_store_name,
    st.s_store_sk,
    st.s_country,
    SUM(sr.sr_return_amt) AS total_returns
  FROM store_returns AS sr
  INNER JOIN store AS st ON sr.sr_store_sk = st.s_store_sk
  GROUP BY st.s_sto

'Max attempts reached. Last error: Binder Error: column "s_store_name" must appear in the GROUP BY clause or must be part of an aggregate function.\nEither add it to the GROUP BY list, or use "ANY_VALUE(s_store_name)" if the exact value of "s_store_name" is not important.'

Execution: FAILED

Execution Time: 73.41 seconds
 ############################### 
CPU times: total: 1.33 s
Wall time: 8min 9s


'Successfully processed 20 questions. Results saved to /tmp/llm\\results_20250508_231153.json'

In [20]:
duckdb.sql(f""" from '{output_dir}/*.json' order by nbr,model """).to_view("results")

┌──────────┬───────┬──────────────────────┬────────────┬──────────────────────┬──────────┬──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [21]:
duckdb.sql(""" from results where error_details is not null""")

┌──────────┬───────┬───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬────────────┬───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [22]:
duckdb.sql(f""" Select model,nbr,result from results order by nbr,model """).show(max_rows=1000)

┌──────────┬───────┬────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────